In [26]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class PositionalEncoding(nn.Module):
    # 위치적 인코딩 할 것 
    # 실제 PE 를 구하는 식이 있음
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # dropout 할 비율을 정하는 것
        pe = torch.zeros(max_len, d_model)
        # 0으로 이루어진 행렬을 max_len X d_model 형태로 만들어 pe 에 넣어둠
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 위치는 0~ max_len 까지 원소를 만들고, max_len 길이의 행으로 변경
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # d_model 까지의 길이를 2 간격으로 숫자를 뽑아놓은 다음, 주어진 공식에 맞게 div_term 을 구함
        pe[:, 0::2] = torch.sin(position * div_term)
        # 짝수에는 sin()를, 홀수에는 cos() 함수를 취한다
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)

In [69]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout = 0.5):
        super(TransformerModel, self).__init__()
        # super(모델명, self).__init__() 으로 한번 명시해줘야 함
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        self.init_weights()
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0,1)
        # torch.ones(sz, sz) : 1로 구성된 행렬을 sz x sz 형태로 만듦
        # torch.ones(sz, sz) == 1 : 각 원소가 1이라면 True, 아니라면 False
        # torch.triu(torch.ones(sz, sz)) == 1 는 행렬의 윗 삼각형 부분만 건져냄(=true 로 만듦)
        # .transpose(0,1) : 행렬 변환
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        # 윗 삼각형은 -inf 로 만들고, 아랫삼각형은 0으로 채워둠
        return mask
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # 인코더의 가중치 초기값은 -initrange ~ initrange
        self.decoder.bias.data.zero_()
        # 디코더의 bias 는 0으로 초기화
        self.decoder.weight.data.uniform_(-initrange, initrange)
        # 디코더의 가중치 값도 동일하게 -initrange ~ initrange
        
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            # mask 가 none 이거나, 길이가 맞지 않을 때
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask) 
        # src 와 src_mask 를 인코더에 넣어 결과물을 받는다
        output = self.decoder(output) # 결과물을 디코더에 넣어 결과물을 얻는다
        return output

        
        

positional encoding : 토큰의 절대적인 포지션을 주입함

In [70]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 우선 dropout 을 할 p 값을 정해준다

        pe = torch.zeros(max_len, d_model)
        # d_model 은 인코더, 디코더에서 정해진 입력과 출력의 크기를 의미
        # 논문에서는 512 개로 한정
        # 5000개의 단어를 d_model 만큼 만들어 pe 행렬 생성
        
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 우선 0~ max_len까지의 수를 만든 후 행렬변환
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # 0~ d_model 까지 2개 차이나도록 만든 후 exp() 을 반영
        # 이후 -4 / d_model 만큼, 입출력 크기 만큼 나누어준다
        
        pe[:, 0::2] = torch.sin(position * div_term)
        # 만들어진 pe 의 짝수열에는 position 의 sin값을 넣고
        
        pe[:, 1::2] = torch.cos(position * div_term)
        # 만들어진 pe 의 홀수열에는 cos 값을 넣는단
        
        pe = pe.unsqueeze(0).transpose(0, 1) # 다시 행렬변환
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        # 그 pe 값을 x 값과 더한 결과를 순전파하는 것으로 사용한다
        return self.dropout(x)

## 데이터 로드하고 배치 만들기

In [60]:
import torchtext
from torchtext.data.utils import get_tokenizer

In [61]:
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"), 
                           init_token='<sos>',
                           eos_token='<eos>',
                           lower=True)

In [63]:
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)

In [64]:
TEXT.build_vocab(train_txt)

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 사용하고 있는 장비가 GPU or CPU 확인

cpu


In [71]:
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # data.examples[0].text 는 띄어쓰기 처럼 문장을 나눈 단어로 구성
    # .numericalize 를 통해 숫자로 변환
    
    # 데이터셋을 bsz파트들로 나눔
    nbatch = data.size(0) // bsz
     
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들)은 잘라낸다
    data = data.narrow(0, 0, nbatch * bsz)
    # narrow 는 주어진 차원에 따라 data 를 잘라내는 역할
    # 0 차원, 0번째 원소부터 nbatch * bsz 길이까지만 data로 가져옴
    
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1).t().contiguous()
    # contiguous() 하면 실제 텐서의 복사본이 만들어짐
    # 이는 narrow, view 등등, 텐서의 메모리를 공유하는 기능과 다르다
    # 덕분에 메모리 상의 순서가 중요한 경우 이 복사 기능을 통해 오류를 방지할 수 있다
    
    return data.to(device)

In [72]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [73]:
# 인스턴스 초기화

ntokens = len(TEXT.vocab.stoi) # 단어 사전의 크기
emsize = 200 # 임베딩 차원
nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
nlayers = 2 # nn.TransformerEncoder 내부의 nn.TransformerEncoderLayer 개수
nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수
dropout = 0.2 # 드랍아웃(dropout) 값
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [77]:
criterion = nn.CrossEntropyLoss()
lr = 5.0
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.95)

import time

def train():
    model.train()   # 모델 학습 시작
    total_loss = 0. # 초기 loss 값은 0으로 셋팅 
    start_time = time.time() # 숫자로 된 현재 시간
    ntokens = len(TEXT.vocab.stoi) # TEXT.vocab 의 내용을 dict 형태로 저장해둔 것, 28871 개
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i) #data.shape : 35 x 20, targets.shape = 700
    
        # 갱신할 함수 모두 0으로 만들어야, backward 호출할 때마다 buffer 에 누적되지 않기 때문
        optimizer.zero_grad() # 역전파 실행하기 전에  옵티마이저 grad 0으로 변환 
        output = model(data) # 모델에 넣어 output 을 반환받음, output.shape = 35, 20, 28871
        loss = criterion(output.view(-1, ntokens), targets) # output 의 가장 오른쪽 열(-1) 과 targets 을 실제로 비교하기
        loss.backward() # 역전파 단계 : 모델의 매개변수에 대한 손실의 변화도 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # 모델의 gradient norm 을 클리핑
        optimizer.step() # step 함수를 호출하면 매개변수가 갱신

        total_loss += loss.item() #loss 내 dictionary 형태로 저장되었던 값을 더해준다
        log_interval = 200 # 로그 보여주는 간격은 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval # 200번에 한번씩 최종 loss 감소율을 보여준다
            elapsed = time.time() - start_time # 현재까지 걸린 시간도 보여준다
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                      'lr {:02.2f} | ms/batch {:5.2f} | '
                      'loss {:5.2f} | ppl {:8.2f}'.format(
                        epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                        elapsed * 1000 / log_interval,
                        cur_loss, math.exp(cur_loss)))
    total_loss = 0
    start_time = time.time()
      
def evaluate(eval_model, data_source):
    eval_model.eval() # 평가 모드 시작
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad(): 
    # torch.no_grad() 로 감싸줌 : 가중치들이 requires_grad = True 이지만 autograd 에서는 추적할 필요없기때문
        for i in range(0, data_source.size(0) - 1, bptt):
            data.targets = get_batch (data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens) # 평가 데이터로 model 을 태워 만든 output 의 가장 오른쪽 열만 가져온다
            total_loss += len(data) * criterion(output_flat, targets).item()   # 데이터 길이 * loss 차이 를 더해준다
    return total_loss / (len(data_source) -1)

In [78]:
# 에포크 내에서 반복
# 만약 검증 오차 validation loss 갖는다면  우리가 지금까지 관찰한 것 중 최적이라면 모데릉ㄹ 저장
# 매 에포크 이후에 학습률을 조절

In [ ]:
best_val_loss = float("inf")
epochs = 3 # 에포크수
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    print(epoch_start_time)
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} |'
    'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss  = val_loss
        best_model  = model

scheduler.step() 
# 매개변수화 해줌
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.95)
# step 은 처음에

1600267540.10833
